In [1]:
import papermill as pm
import torch
import mlflow
from datasets import TestImageNetDataset
import tqdm

from torch.utils.data import DataLoader

from torchvision.models import ResNet50_Weights

/root/miniconda3/envs/mlrc_2023/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/root/miniconda3/envs/mlrc_2023/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/root/miniconda3/envs/mlrc_2023/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# Default Parameters
run_id = "5363b9a6f6954354b7bb68d535b5ea88"

test_data_path = "../data/ImageNet/ILSVRC/Data/CLS-LOC/val/"
test_data_labels_path = "../data/ImageNet/LOC_val_solution.csv"

label_mapping_path = "../data/ImageNet/LOC_synset_mapping.txt"

resnet50_weights = ResNet50_Weights.DEFAULT

preprocess = resnet50_weights.transforms()
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# Parameters
run_id = "06f55f328f4d4aee8f45d5f2e1ba9a31"
test_data_path = "../data/ImageNet/ILSVRC/Data/CLS-LOC/val/"
test_data_labels_path = "../data/ImageNet/LOC_val_solution.csv"
label_mapping_path = "../data/ImageNet/LOC_synset_mapping.txt"


In [4]:
# load in saved model
with mlflow.start_run(run_id=run_id) as run:
    teacher_model_uri = f"runs:/{run.info.run_id}/teacher"
    teacher = mlflow.pytorch.load_model(teacher_model_uri)
    
    student_model_uri = f"runs:/{run.info.run_id}/student"
    student = mlflow.pytorch.load_model(student_model_uri)
    
    independent_student_model_uri = f"runs:/{run.info.run_id}/independent_student"
    independent_student = mlflow.pytorch.load_model(independent_student_model_uri)

In [5]:

# label_mapping = {}
# with open(label_mapping_path) as f:
#     reader = csv.reader(f)
#     for mapping in reader:
#         mapping = mapping[0].split() + mapping[1:]
#         id = mapping[0]
#         label = ', '.join(mapping[1:])
#         label_mapping[id] = label
        
                

In [6]:
test_dataset = TestImageNetDataset(test_data_path, test_data_labels_path, label_mapping_path, preprocess)
test_dataloader = DataLoader(test_dataset, batch_size=80, shuffle=False)

In [7]:
def evaluate(student, teacher, independent_student, test_dataloader, device):
    """
    - teacher: The pretrained model used to help the student model learn
    - student: The smaller, untrained model that uses the teacher's output as an additional label
    - independent_student: The smaller, untrained model that doesn't use the teacher's output as an additional label
    - test_dataloader: Dataloader for test set
    - device: Device to run training
    """
    teacher.eval()
    teacher.to(device)
    student.train()
    student.to(device)
    independent_student.eval()
    independent_student.to(device)
    
    teacher_correct_predictions = 0
    student_correct_predictions = 0
    independent_student_correct_predictions = 0
    total_samples = 0
    with torch.no_grad():
        
        for inputs, labels in tqdm.tqdm(test_dataloader):
            inputs, labels = inputs.to(device), labels.to(device)

            teacher_predictions = torch.argmax(teacher(inputs), axis=1)
            student_predictions = torch.argmax(student(inputs), axis=1)
            independent_student_predictions = torch.argmax(independent_student(inputs), axis=1)

            teacher_correct_predictions += (teacher_predictions == labels).sum().item()
            student_correct_predictions += (student_predictions == labels).sum().item()
            independent_student_correct_predictions += (independent_student_predictions == labels).sum().item()

            total_samples += labels.size().numel()


    teacher_accuracy = teacher_correct_predictions / total_samples
    student_accuracy = student_correct_predictions / total_samples
    independent_student_accuracy = independent_student_correct_predictions / total_samples

    print("Teacher Accuracy: ", teacher_accuracy)
    print("Student Accuracy: ", student_accuracy)
    print("Independent Student Accuracy: ", independent_student_accuracy)
    
    with mlflow.start_run(run_id=run_id) as run:
        mlflow.log_metric("teacher_accuracy", teacher_accuracy)
        mlflow.log_metric("student_accuracy", student_accuracy)
        mlflow.log_metric("independent_student_accuracy", independent_student_accuracy)


evaluate(student, teacher, independent_student, test_dataloader, device)

  0%|          | 0/625 [00:00<?, ?it/s]

  0%|          | 1/625 [00:04<43:08,  4.15s/it]

  0%|          | 2/625 [00:08<44:00,  4.24s/it]

  0%|          | 3/625 [00:12<44:14,  4.27s/it]

  1%|          | 4/625 [00:17<45:13,  4.37s/it]

...

 45%|████▌     | 284/625 [20:58<24:14,  4.27s/it]

 46%|████▌     | 285/625 [21:02<23:41,  4.18s/it]

 46%|████▌     | 286/625 [21:06<23:47,  4.21s/it]

 46%|████▌     | 287/625 [21:10<23:20,  4.14s/it]

 46%|████▌     | 288/625 [21:15<23:24,  4.17s/it]